In [9]:
import sqlite3
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
pathname = "/Users/kabbo/Desktop/marcy/Mod 3/Supernova Project/data/themepark.db"
connection = sqlite3.connect(pathname)
cursor = connection.cursor()

In [10]:
# Set a consistent, professional style for the plots
sns.set_style("whitegrid")
sns.set_palette("bright")
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

In [4]:
# --- Visualization 1: Top Attractions by Average Wait Time ---

query_attraction_waits = """
SELECT
    a.attraction_name,
    AVG(e.wait_minutes) AS average_wait
FROM fact_ride_events e
JOIN dim_attraction a ON e.attraction_id = a.attraction_id
WHERE e.wait_minutes IS NOT NULL
GROUP BY a.attraction_name
ORDER BY average_wait DESC
LIMIT 10;
"""
df_waits = pd.read_sql_query(query_attraction_waits, connection)

plt.figure()
ax = sns.barplot(x='average_wait', y='attraction_name', data=df_waits, orient='h')
plt.xlabel("Average Wait Time (Minutes)")
plt.ylabel("Attraction")
plt.title("Top Attractions by Average Wait Time", fontsize=16, fontweight='bold')
for container in ax.containers:
    ax.bar_label(container, fmt='%.1f min', padding=3)
plt.savefig("/Users/kabbo/Desktop/marcy/Mod 3/Supernova Project/figures/top_attractions_by_wait_time.png", bbox_inches="tight")
plt.close()

In [10]:
# --- Visualization 2: Annotated Bar Chart ---

query_wait_tolerance = """
WITH attraction_metrics AS (
  SELECT a.attraction_id, a.attraction_name, a.wait_tolerance,
         AVG(e.wait_minutes) AS avg_wait,
         AVG(e.satisfaction_rating) AS avg_satisfaction
  FROM dim_attraction a
  JOIN fact_ride_events e ON a.attraction_id = e.attraction_id
  WHERE e.wait_minutes IS NOT NULL AND e.satisfaction_rating IS NOT NULL
  GROUP BY a.attraction_id, a.attraction_name, a.wait_tolerance
)
SELECT * FROM attraction_metrics ORDER BY avg_wait DESC;
"""
df_tolerance = pd.read_sql_query(query_wait_tolerance, connection)

# Create the horizontal bar chart, color-coded by wait_tolerance
plt.figure()
ax = sns.barplot(
    data=df_tolerance,
    x='avg_wait',
    y='attraction_name',
    hue='wait_tolerance',
    palette={'High': '#2ca02c', 'Standard': '#1f77b4', 'Low': '#d62728'}, # Green, Blue, Red
    dodge=False # Prevents bars from splitting
)

# Add annotations for both wait time and satisfaction to each bar
for index, row in df_tolerance.iterrows():
    # Place the wait time text at the end of the bar
    ax.text(row.avg_wait + 0.5, index, f'{row.avg_wait:.1f} min',
            color='black', ha='left', va='center', fontsize=11)
    # Place the satisfaction score text inside the bar for context
    ax.text(2, index, f'Satisfaction: {row.avg_satisfaction:.2f}',
            color='white', ha='left', va='center', fontsize=11, fontweight='bold')


# Final styling for clarity
ax.set_xlabel("Average Wait Time (Minutes)", fontsize=12)
ax.set_ylabel("Attraction", fontsize=12)
ax.set_title("Wait Times & Satisfaction by Tolerance Tier", fontsize=18, fontweight='bold')
ax.legend(title="Wait Tolerance Category")
# Remove x-axis ticks for a cleaner look since we have labels
ax.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)

plt.savefig("/Users/kabbo/Desktop/marcy/Mod 3/Supernova Project/figures/wait_tolerance_barchart.png", bbox_inches="tight")
plt.close()


In [11]:
# --- 5. Visualization 3: Financial Impact of High Wait Times ---
query_financial_impact = """
SELECT
    CASE
        WHEN high_wait_flag = 1 THEN 'Experienced a High Wait'
        ELSE 'Did Not Experience a High Wait'
    END AS wait_experience,
    AVG(spend_per_person) / 100.0 AS avg_spend_per_person_dollars
FROM fact_visits
WHERE spend_per_person IS NOT NULL
GROUP BY high_wait_flag;
"""
df_financial_impact = pd.read_sql_query(query_financial_impact, connection)

# Define a custom color palette to clearly distinguish the two groups
custom_palette = {
    "Did Not Experience a High Wait": "#2ca02c",  # Green for positive
    "Experienced a High Wait": "#d62728"      # Red for negative
}

plt.figure()
# Add the 'palette' argument to the barplot function
ax = sns.barplot(
    x='wait_experience',
    y='avg_spend_per_person_dollars',
    data=df_financial_impact,
    palette=custom_palette,
    hue='wait_experience',
    legend=False
)

plt.xlabel("Guest Experience")
plt.ylabel("Average Spend Per Person ($)")
plt.title("Guests Spend Less After Experiencing a High Wait", fontsize=16, fontweight='bold')
for container in ax.containers:
    ax.bar_label(container, fmt='$%.2f', padding=3)
plt.savefig("/Users/kabbo/Desktop/marcy/Mod 3/Supernova Project/figures/financial_impact_of_waits.png", bbox_inches="tight")
plt.close()

In [12]:
connection.close()